In [1]:
import os
import json
from tqdm import tqdm
import os
import json
import pandas as pd
from tqdm import tqdm
import sys
sys.path.append("D:/_WangKe/ganvana")
from imgs.get_imgs import imgurl2path, get_img_list, clean_imgurl
from show.show_goods import id2show_goods

In [19]:
tags_cate1_cate2 = json.load(open("D:/_WangKe/scikkk.github.io/projects/ganvana/tags_cate1_cate2.json", "r", encoding="utf-8"))
tags_cate1_cate2["tags"] = {item['id']: item for item in tags_cate1_cate2["tags"]}
id_cates = {item['id']: item for item in tags_cate1_cate2["cate1List"]+tags_cate1_cate2["cate2List"]}
name_cates = {item['name']: item for item in tags_cate1_cate2["cate1List"]+tags_cate1_cate2["cate2List"]}

In [20]:
cates = {}
for cate1 in tags_cate1_cate2["cate1List"]:
    for cate2 in tags_cate1_cate2["cate2List"]:
        if cate2["father"] == cate1["id"]:
            if cate1["name"] not in cates:
                cates[cate1["name"]] = []
            cates[cate1["name"]].append(cate2["name"])
with open("D:/_WangKe/scikkk.github.io/projects/ganvana/cates.json", "w", encoding="utf-8") as f:
    json.dump(cates, f, ensure_ascii=False, indent=4)

In [4]:
import datetime

def convert_timestamp_to_datetime(timestamp):
    timestamp_in_seconds = timestamp / 1000.0
    dt = datetime.datetime.fromtimestamp(timestamp_in_seconds)
    return dt

In [5]:
def load_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in tqdm(f):
            data.append(json.loads(line))
    return data


In [6]:

def mall_jsonl2csv(jsonl_file):
    vacant_list = []
    data = load_jsonl(jsonl_file)
    headers = [     
            'page', 'id', 
            'tag', 
            'group', 
            'cate1', 'cate1_name',
            'cate2', 'cate2_name',
            'status', 
            'name', 'lat_name', 'sn', 
            'author', 
            'price', 'usd_price', 'profit_rate',
            'amount', 'size', 'condition', 'area', 
            'thumb', 'thumb_big', 
            'desc', 'descEn', 'content', 'created_at', 'updated_at', 'indexShow', 
            'cate_name', 'cate_lat_name',  
            'cls_ph', 'cls_cl1', 'cls_cl2', 'cls_cl3', 
            'cls_or1', 'cls_or2', 'cls_or3', 'cls_or4', 'cls_or5', 
            'cls_fa1', 'cls_ch', 
               ]
    for entry in tqdm(data, desc="Processing JSONL"):
        if len(entry) <= 1:
            vacant_list.append(entry["id"])
            continue
        # classification
        classification = ["ph", "cl1", "cl2", "cl3", "or1", "or2", "or3", "or4", "or5", "fa1", "ch"]
        if "classification" not in entry or not entry["classification"]:
            entry["classification"] = {}
        for c in classification:
            entry[f"cls_{c}"] = entry["classification"].get(c, '')
        del entry["classification"]
        # cate1, cate2
        cate1 = min(entry.get("cate1", 0), entry.get("cate2", 0))
        cate2 = max(entry.get("cate1", 0), entry.get("cate2", 0))
        entry["cate1"], entry["cate2"] = cate1, cate2
        if cate1 in id_cates:
            entry["cate1_name"] = id_cates[cate1]["name"]
        else:
            entry["cate1_name"] = ''
        if cate2 in id_cates:
            entry["cate2_name"] = id_cates[cate2]["name"]
            entry["cate1_name"] = id_cates[id_cates[cate2]["father"]]["name"]
        else:
            entry["cate2_name"] = ''

        if "name" in entry and entry["name"] and not entry["cate2_name"]:
            for candidates in ["亚纳特螺科","德亚奇螺科", "豆蜗牛科", "小豆螺科", "亚轮螺科", "塔格螺科", "薄泥蜗牛科", "拟阿勇螺科", "吉奥蜗牛科", "管螺科"]:
                if candidates in entry["name"] or candidates[:-1] in entry["name"]:
                    entry["cate2_name"] = name_cates[candidates]["name"]
                    entry["cate1_name"] = id_cates[name_cates[candidates]["father"]]["name"]
                    # print(entry["cate1_name"], entry["cate2_name"], entry["name"])
                    break
        
        if entry.get("price", 0) and entry.get("usd_price", 0):
            entry["profit_rate"] = round((entry["price"] - entry["usd_price"]) / entry["usd_price"], 2)
        if 'created_at' in entry and isinstance(entry['created_at'], int):
            entry['created_at'] = convert_timestamp_to_datetime(entry['created_at'])
        if 'updated_at' in entry and isinstance(entry['updated_at'], int):
            entry['updated_at'] = convert_timestamp_to_datetime(entry['updated_at'])
    # deduplicate
    data = {entry["id"]: entry for entry in data}
    data = list(data.values())
    df = pd.DataFrame(data, columns=headers)
    return df

In [7]:
getGoodsInfo = "D:/_WangKe/scikkk.github.io/projects/ganvana/mall/getGoodsInfo.jsonl"
data = mall_jsonl2csv(getGoodsInfo)

0it [00:00, ?it/s]

151613it [00:02, 51023.66it/s]
Processing JSONL: 100%|██████████| 151613/151613 [00:01<00:00, 89227.59it/s]


In [8]:
data[data['id'] == 1]

,page,id,tag,group,cate1,cate1_name,cate2,cate2_name,status,name,...,cls_cl1,cls_cl2,cls_cl3,cls_or1,cls_or2,cls_or3,cls_or4,cls_or5,cls_fa1,cls_ch
0,NaN,1,1.0,2.0,3.0,海生腹足纲,209.0,蟹守螺科,-1.0,未定螺科,...,,,,,,,,,,


In [9]:
def ids2goods_list(idxs, max_num=1000):
    goods_list = []
    for idx in idxs[:max_num]:
        goods_list.append(id2show_goods(idx))
    out = {"list":goods_list,
    "pages": {
      "amount": len(goods_list),
      "now_page": "1",
      "all_page": 1
    }}
    with open("D:/_WangKe/scikkk.github.io/projects/ganvana/show/goods_list.json", "w", encoding="utf-8") as f:
        json.dump(out, f, ensure_ascii=False, indent=4)
    

In [15]:
# show data info
data = data.sort_values(by="price", ascending=False)
ids = data[data['name'].str.contains("芋螺", na=False)]["id"].tolist()
ids2goods_list(ids)


In [21]:
# show data info
data = data.sort_values(by="price", ascending=False)
c1 = name_cates["坚螺科"]['id']
c2 = name_cates["坚螺科"]['id']
ids = data[(data['cate1'] == c1) | (data['cate2'] == c2)]["id"].tolist()
ids2goods_list(ids)
